In [1]:
import numpy as np
import glob
from src import parseTrackletXML as pt_XML

In [2]:
lidar_points = glob.glob('./velodyne_points/data/*.bin')


print(len(lidar_points))

154


In [3]:
def load_tracklet(xml_path, n_frame):
    tracklets = pt_XML.parseXML(xml_path)
    objects = []  #grouped by frames
    
    # refered to parseTrackletXML.py example fuction
    # loop over tracklets
    for tracklet in tracklets:
        
        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        trackletBox = np.array([ # in velodyne coordinates around zero point and without orientation yet\
        [-l/2, -l/2,  l/2, l/2, -l/2, -l/2,  l/2, l/2], \
        [ w/2, -w/2, -w/2, w/2,  w/2, -w/2, -w/2, w/2], \
        [ 0.0,  0.0,  0.0, 0.0,    h,     h,   h,   h]])
        
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:

            # determine if object is in the image; otherwise continue
            if truncation not in (TRUNC_IN_IMAGE, TRUNC_TRUNCATED):
                continue

            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]   # other rotations are 0 in all xml files I checked
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([\
              [np.cos(yaw), -np.sin(yaw), 0.0], \
              [np.sin(yaw),  np.cos(yaw), 0.0], \
              [        0.0,          0.0, 1.0]])
            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8,1)).T

            # calc yaw as seen from the camera (i.e. 0 degree = facing away from cam), as opposed to 
            #   car-centered yaw (i.e. 0 degree = same orientation as car).
            #   makes quite a difference for objects in periphery!
            # Result is in [0, 2pi]
            x, y, z = translation
            yawVisual = ( yaw - np.arctan2(y, x) ) % twoPi
        
        
    
xml_path = "./tracklet_labels.xml"    
load_tracklet(xml_path, len(lidar_points))

parsing tracklet file ./tracklet_labels.xml
file contains 15 tracklets
loaded 15 tracklets


AttributeError: module 'itertools' has no attribute 'izip'

In [16]:
def projectToImage(points_3d, K):
    points_2d = np.dot(K, points_3d[:3,:])
    points_2d[0,:]=points_2d[0,:]/points_2d[2,:]
    points_2d[1,:]=points_2d[1,:]/points_2d[2,:]
    return points_2d[:2,:]
